In [ ]:
import os
from river import datasets
from confluent_kafka import Producer,Consumer
import certifi
import time
import json

In [ ]:
user= os.environ['kafka_username']
password= os.environ['kafka_password']
bsts= os.environ['kafka_bootstrap_servers']

In [ ]:
topic = 'malicious_url_events'

In [ ]:
conf = {'bootstrap.servers': bsts,
            'sasl.mechanism': 'PLAIN',
            'security.protocol': 'SASL_SSL',
            'ssl.ca.location': certifi.where(),
            'sasl.username': user,
            'sasl.password': password,
            'batch.num.messages': 2048,
            #'queue.buffering.max.messages': 100,
            'linger.ms': 100,
            'client.id': 'producer-icde-2023'}
producer = Producer(conf)    

In [ ]:
max_size=1000000
dataset = datasets.MaliciousURL()
data = dataset.take(max_size)


In [ ]:
cnt = 0
st = time.time()
abs_st = time.time()
for f, y in data:
    cnt = cnt + 1    
    d = {}
    d['f']=f
    d['y']=str(y).lower()
    d['st']=time.time()  
            
    v= json.dumps(d).encode('utf-8')
    try:
        producer.produce(topic, value=v, key=str(cnt))
    except:
      print(f'Queue full, flushing {cnt}')
      producer.flush()
      producer.produce(topic, value=v, key=str(cnt))
    if cnt%10000==0:           
        end = time.time()
        print(f'flushing count - {cnt}, time taken in seconds- {end-st} ')        
        producer.flush()
        st = time.time()        
producer.flush()
end = time.time()
print(f'final flushing count - {cnt}, time taken in seconds- {end-abs_st} ')        

In [ ]:
from river import metrics
auc = metrics.ROCAUC()
f1 = metrics.F1()
recall = metrics.MicroRecall()
features_consumer_conf = {'bootstrap.servers': KAFKA_BOOTSTRAP_SERVERS,
                     'sasl.username': KAFKA_USER_NAME,
                     'sasl.password': KAFKA_PASSWORD,
                     'sasl.mechanism': 'PLAIN',
                     'security.protocol': 'SASL_SSL',
                     'ssl.ca.location': certifi.where(),
                     'group.id': 'prediction-grp',
                     'enable.auto.commit': True,
                     'auto.offset.reset': 'latest'}
features_consumer = Consumer(features_consumer_conf)    


In [ ]:
import statistics
durs = []
mem_usage = []
cnt = 0
while(cnt<1000000):
    msg = consumer.poll(timeout=0.1)
    if msg is None: continue
    if msg.error():
        if msg.error().code() == KafkaError._PARTITION_EOF:
                sys.stderr.write('%% %s [%d] reached end at offset %d\n' %
                                         (msg.topic(), msg.partition(), msg.offset()))
        else:            
            cnt = cnt + 1
            message = loads(msg.value().decode("utf-8"))
                                new_message['y']=(message['y']=='true')     
            score = message['score']
            y = message['y']
            durs.append(message['duration'])
            mem_usage.append(message['mem_usage'])
            auc = auc.update(y, score)
            f1 = f1.update(y, score)
            recall = recall.update(y, score)
            

In [ ]:
mean = statistics.mean(durs)
median = statistics.median(durs)
max1 = max(durs)
min2 = min(durs)
total_records = len(durs)
memory_usage = statistics.mean(mem)
print(auc)
print(f1)
print(recall)